<a href="https://colab.research.google.com/github/BAfsharmanesh/CS8945-project/blob/main/manifold_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install gdown
!gdown --folder "https://drive.google.com/drive/folders/1IUaQJgQY1i9TZDWnY9x_eE8HBBSu22T_?usp=drive_link" -O /content/dataset

Retrieving folder contents
Retrieving folder 1WCUV6adZ8K-OQrb0qYLrirlJoypAzfaj rep-llama2-7b
Processing file 1cQj67jvxtmX8MMOe75EVlNfZ3gNHTcVw rep-model.layers.0.post_attention_layernorm.npy
Processing file 1V7XV118zYQebVDiI3Rx-AlyLduBK4v94 rep-model.layers.1.post_attention_layernorm.npy
Processing file 1wDUZA_XrmIH8xWGsZ4683e6wC_kks5Zi rep-model.layers.2.post_attention_layernorm.npy
Processing file 1bUz1wBErSSEmt3Lstitx6Bftpa_rRMxM rep-model.layers.3.post_attention_layernorm.npy
Processing file 1A7lSiuYKEPvM_2O_KDonObE9fXxbJIcY rep-model.layers.4.post_attention_layernorm.npy
Processing file 1r8o4udUnjDhmCwHHWAm6ueC4i-n2EoUz rep-model.layers.5.post_attention_layernorm.npy
Processing file 1agkLK4egDgpQ4S3rJUcDb6M8F0Fld_XY rep-model.layers.6.post_attention_layernorm.npy
Processing file 1i0MsXSwmam8MtX1AHgIB5ozrDqtGcQNk rep-model.layers.7.post_attention_layernorm.npy
Processing file 1AeGUUyutVLhWzKJjBQOvlMo1fM4t1Lvy rep-model.layers.8.post_attention_layernorm.npy
Processing file 1B4NxZ4xv

In [ ]:
import os
import numpy as np

# Directory containing the .npy files
folder_path = '/content/dataset/rep-llama2-7b'

# List all files in the directory
all_files = os.listdir(folder_path)

# Filter for .npy files
npy_files = [f for f in all_files if f.endswith('.npy')]

# Dictionary to store the loaded data
loaded_data = {}

# Load each .npy file
for file_name in npy_files:
    file_path = os.path.join(folder_path, file_name)
    if 'layers' in file_name:
      layer_number = file_name.split('.')[2]
      loaded_data[layer_number] = np.load(file_path)
    else:
      loaded_data['norm'] = np.load(file_path)

# You can access the loaded data using the file names as keys
# For example, to access the data from 'rep-model.norm.npy':
# print(loaded_data['rep-model.norm.npy'].shape)

print(f"Loaded {len(loaded_data)} .npy files.")

Loaded 33 .npy files.


In [ ]:
loaded_data["0"].shape

(11855, 4096)